In [1]:
## 결측치 보간 코드 ## 

import pandas as pd
from sklearn.impute import SimpleImputer

# 1. 데이터 로드
train_df = pd.read_csv('X_train.csv', index_col=0)
test_df = pd.read_csv('X_test.csv', index_col=0)   # ← 여기 원래 X_train으로 되어 있던 부분 수정
df_meta = pd.read_excel('./updated_dataframe.xlsx')

# 2. 메타데이터에서 category 변수 목록 추출
categorical_variables = df_meta[df_meta['data type'] == 'category']['variable'].tolist()

# 3. train_df 기준으로 실제 존재하는 범주형 컬럼만 교집합
intersected_categorical_variables = list(set(train_df.columns) & set(categorical_variables))

# 4. 범주형 변수 결측치: train 기준 최빈값으로 채우고, 같은 값으로 test도 채움
for col in intersected_categorical_variables:
    # train에서 mode 계산 (여러 개면 첫 번째 사용)
    mode_value = train_df[col].mode(dropna=True)
    if len(mode_value) == 0:
        # mode가 없을 정도로 데이터가 비어 있으면 그냥 넘어가기
        continue
    mode_value = mode_value.iloc[0]

    # train, test에 동일하게 적용
    train_df[col] = train_df[col].fillna(mode_value)
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna(mode_value)

# 5. 수치형 변수 목록 선정 (범주형 제외 + dtype이 숫자인 것만)
numeric_candidates = [c for c in train_df.columns if c not in intersected_categorical_variables]
numeric_cols = [
    c for c in numeric_candidates
    if pd.api.types.is_numeric_dtype(train_df[c])
]

# 6. SimpleImputer로 수치형 변수 중앙값 대치 (IterativeImputer보다 훨씬 빠름)
num_imputer = SimpleImputer(strategy='median')

# train에 fit, train/test 모두 transform
train_df[numeric_cols] = num_imputer.fit_transform(train_df[numeric_cols])
test_df[numeric_cols] = num_imputer.transform(test_df[numeric_cols])

# 7. 결과 저장
train_df.to_csv('X_train_imputed.csv', index=True)
test_df.to_csv('X_test_imputed.csv', index=True)

In [3]:
## 모델 학습 및 예측 코드 ## 

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# 1. 데이터 로드 ----------------------------------------------------
X_train = pd.read_csv('X_train_imputed.csv', index_col=0)
y_train = pd.read_csv('Y_train.csv', index_col=0)
X_test = pd.read_csv('X_test_imputed.csv', index_col=0)
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)

# 2. train X, y 인덱스 정렬 (혹시라도 어긋나 있을 경우를 대비) ----------
X_train, y_train = X_train.align(y_train, join='inner', axis=0)
X_test, sample_submission = X_test.align(sample_submission, join='inner', axis=0)

# 3. RandomForest 모델 정의 및 학습 ---------------------------------
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)

# 4. X_test_imputed에 대해 예측 -------------------------------------
pred_prob = rf.predict_proba(X_test)[:,1]
sample_submission['HE_D3_label'] = pred_prob

# 5. 최종 제출 파일 저장 ---------------------------------------------
sample_submission.to_csv('submission.csv', index=True)

print("학습/예측 완료: 'submission.csv' 파일이 생성되었습니다.")

/opt/homebrew/anaconda3/envs/team_project/lib/python3.12/site-packages/sklearn/base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


학습/예측 완료: 'submission.csv' 파일이 생성되었습니다.
